In [1]:
from firedrake import *
from firedrake.pyplot import *
import matplotlib.pyplot as plt
import numpy as np
from firedrake.cython import dmcommon
from firedrake.mesh import plex_from_cell_list

In [2]:
mesh = Mesh("../meshes/ITER.msh", dim = 2, distribution_parameters={"partition": False}, reorder = True)
mesh.init()

In [14]:
# Define a function space as tool to access nodes and cells:
V_tool = FunctionSpace(mesh, "CG", 1)

# Fill "dof_coords" vector with the coordinates of the boundary nodes of the 2D mesh
coord_func = Function(VectorFunctionSpace(mesh, "CG", 1)).interpolate(as_vector(SpatialCoordinate(mesh)))
dofs = DirichletBC(V_tool, 0.0, "on_boundary").nodes
dof_coords = coord_func.dat.data_ro[dofs]

In [15]:
display(dof_coords)
# It shows that boundary dofs are not ordered.

array([[ 7.00500000e+00,  1.00000000e+01],
       [ 8.00428571e+00,  1.00000000e+01],
       [ 6.00571429e+00,  1.00000000e+01],
       [ 9.00357143e+00,  1.00000000e+01],
       [ 5.00642857e+00,  1.00000000e+01],
       [ 1.00028571e+01,  1.00000000e+01],
       [ 1.10021429e+01,  1.00000000e+01],
       [ 4.00714286e+00,  1.00000000e+01],
       [ 1.20014286e+01,  1.00000000e+01],
       [ 1.30007143e+01,  1.00000000e+01],
       [ 1.40000000e+01,  1.00000000e+01],
       [ 1.40000000e+01,  9.00000000e+00],
       [ 3.00785714e+00,  1.00000000e+01],
       [ 1.40000000e+01,  8.00000000e+00],
       [ 2.00857143e+00,  1.00000000e+01],
       [ 1.00928571e+00,  1.00000000e+01],
       [ 1.40000000e+01,  7.00000000e+00],
       [ 1.00000000e-02,  1.00000000e+01],
       [ 1.00000000e-02,  9.00000000e+00],
       [ 1.40000000e+01,  6.00000000e+00],
       [ 1.00000000e-02,  8.00000000e+00],
       [ 1.40000000e+01,  5.00000000e+00],
       [ 1.00000000e-02,  7.00000000e+00],
       [ 1.

In [ ]:
facets = mesh.topology.exterior_facets.facets
boundary_cells = mesh.topology.exterior_facets.facet_cell_map.values
#display(facets)            # indici delle faces
display(len(facets))        
#display(boundary_cells)
display(len(boundary_cells)) # indici delle celle al bordo

# Meglio usare direttamente il vettore dei dof_coords:

68

68

In [ ]:
# For each dof in the boundary:
#   - identify the two closest nodes;
#   - introduce one node at distance h/3;
#   - include the two segments in cell.

        # Add a point in the direction i->j at 1/3 distance:
        #new_dof = 2/3*dof_coords[i] + 1/3*dof_coords[k]
        #new_idx = len(dof_coords_list)-1
        #dof_coords_list.append(new_dof.tolist())

In [16]:
def fill_dist_matrix(vect):
    # Fills the distance matrix between points
    n = len(vect)
    dist_matrix = []

    for i in range(n):
        row = [np.zeros(n)]
        for j in range(n):
            row[j] = np.linalg.norm(vect[i]-vect[j])
        row[i] = np.inf
        dist_matrix.append(row)

    return dist_matrix

In [54]:
dof_coords = coord_func.dat.data_ro[dofs]

# Fill the segments = cells of the 1D boundary mesh:
n = len(dof_coords)
segments = set()   # format "set" to ignore duplicates

for i in range(n):

    # Compute the distance with the others dofs:
    dist = np.zeros(n)
    for j in range(n):
        dist[j] = np.linalg.norm(dof_coords[i]-dof_coords[j])
    dist[i] = np.inf    # set distance with itself = inf

    # Identify the indexes of the two neighbouring dofs of dof_coords[i]
    neighbour_dofs = np.argsort(dist)[:2]

    for k in neighbour_dofs:
        segments.add(tuple(sorted([i,k])))

# Convert "segments" in array:
segments = np.array(list(segments))


In [55]:
display(len(dof_coords))
display(segments)

68

array([[32, 34],
       [58, 59],
       [18, 20],
       [35, 36],
       [44, 51],
       [50, 61],
       [61, 64],
       [46, 51],
       [ 0,  2],
       [ 8,  9],
       [23, 25],
       [49, 50],
       [40, 47],
       [34, 37],
       [65, 67],
       [17, 18],
       [ 1,  3],
       [19, 21],
       [28, 30],
       [36, 40],
       [45, 52],
       [55, 56],
       [37, 48],
       [16, 19],
       [ 6,  8],
       [15, 17],
       [24, 26],
       [33, 35],
       [41, 42],
       [64, 65],
       [59, 60],
       [ 5,  6],
       [20, 22],
       [29, 31],
       [43, 54],
       [46, 47],
       [14, 15],
       [39, 44],
       [ 0,  1],
       [ 9, 10],
       [ 2,  4],
       [10, 11],
       [11, 13],
       [13, 16],
       [30, 32],
       [63, 66],
       [ 7, 12],
       [56, 57],
       [25, 27],
       [42, 43],
       [26, 28],
       [ 4,  7],
       [12, 14],
       [ 3,  5],
       [21, 23],
       [41, 53],
       [38, 39],
       [38, 45],
       [22, 24

In [61]:
plex = plex_from_cell_list(1, segments, dof_coords, comm=mesh.comm)
m = Mesh(plex, dim=1, reorder = True)
m.init()

In [62]:
V = FunctionSpace(mesh,"P",1)
Q = FunctionSpace(m, "P", 1)

In [65]:
x,y = SpatialCoordinate(mesh)
f_2D = Function(V).interpolate(y)

# Interpolation on boundary mesh:
f_1D = Function(Q).zero()
boundary_nodes = dofs = DirichletBC(V_tool, 0.0, "on_boundary").nodes
f_1D.dat.data[:] = f_2D.dat.data_ro[boundary_nodes]


In [66]:
f_1D.dat.data[:]

array([ 1.0000000e+01,  1.0000000e+01,  1.0000000e+01,  1.0000000e+01,
        1.0000000e+01,  1.0000000e+01,  1.0000000e+01,  1.0000000e+01,
        1.0000000e+01,  1.0000000e+01,  1.0000000e+01,  9.0000000e+00,
        1.0000000e+01,  8.0000000e+00,  1.0000000e+01,  1.0000000e+01,
        7.0000000e+00,  1.0000000e+01,  9.0000000e+00,  6.0000000e+00,
        8.0000000e+00,  5.0000000e+00,  7.0000000e+00,  4.0000000e+00,
        6.0000000e+00,  3.0000000e+00,  5.0000000e+00,  2.0000000e+00,
        4.0000000e+00,  1.0000000e+00,  3.0000000e+00, -1.2290613e-11,
        2.0000000e+00, -1.0000000e+00,  1.0000000e+00, -2.0000000e+00,
       -3.0000000e+00,  1.2290613e-11, -1.0000000e+01, -9.0000000e+00,
       -4.0000000e+00, -9.0000000e+00, -1.0000000e+01, -1.0000000e+01,
       -8.0000000e+00, -1.0000000e+01, -6.0000000e+00, -5.0000000e+00,
       -1.0000000e+00, -1.0000000e+01, -1.0000000e+01, -7.0000000e+00,
       -1.0000000e+01, -8.0000000e+00, -1.0000000e+01, -2.0000000e+00,
      